In [16]:
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO('fire.pt')

In [ ]:
# Define the data
data = {
    'degree': [55],
    'd1': [d_values[0],d_values[1],d_values[2],d_values[3],d_values[4]]
}

# Define the value for num
num = 1

# Function to determine the y-coordinate of the black line based on d1 value
def get_black_line_y(d1_value, green_lines, frame_height):
    if d1_value == 0:
        return frame_height  # Black line at the bottom
    if d1_value < green_lines[0][1]:
        return frame_height - int((green_lines[0][0] - (frame_height - d1_value * (green_lines[0][0] - green_lines[-1][0]) / green_lines[0][1])) - frame_height)
    for i in range(len(green_lines) - 1):
        if green_lines[i][1] < d1_value <= green_lines[i + 1][1]:
            return green_lines[i][0] - int((green_lines[i][0] - green_lines[i + 1][0]) * (d1_value - green_lines[i][1]) / (green_lines[i + 1][1] - green_lines[i][1]))
    return green_lines[0][0]  # Above the highest green line

# Function to determine the base line label based on black lines
def get_base_line_label(y_bottom, black_lines, num, opposite):
    # Ensure 'opposite' is a valid float value
    try:
        opposite = float(opposite)  # Ensure it's a float
    except (ValueError, TypeError):
        opposite = 0.0  # Default to 0.0 if not a valid number

    # Sort black lines by y-coordinate
    black_lines = sorted(black_lines, key=lambda x: x[0], reverse=True)

    # Determine the range
    for i in range(len(black_lines) - 1):
        if black_lines[i + 1][0] < y_bottom <= black_lines[i][0]:
            return f"Base line {(i + 1) * num + opposite:.2f} to {(i + 2) * num + opposite:.2f}"
    
    # Special cases
    if y_bottom > black_lines[0][0]:
        return f"Base line 0 to {num + opposite:.2f}"
    if y_bottom <= black_lines[-1][0]:
        return f"Base line {len(black_lines) * num + opposite:.2f} to top"
    
    # Default case if none of the above conditions are met
    return f"Base line 0 to {num + opposite:.2f}"

In [ ]:
# Open the video capture
cap = cv2.VideoCapture(0)

# Loop through the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Ensure the frame is not mirrored
    frame = cv2.flip(frame, 1)  # This flips the image horizontally; if you see mirroring, remove this line.

    # Apply YOLO model to the frame
    results = model(frame)
    detections = results[0].boxes.xyxy.cpu().numpy()  # Get the bounding boxes

    # Get frame dimensions
    frame_height, frame_width = frame.shape[:2]
    cell_size = min(frame_width, frame_height) // 10  # 10 cells in each dimension
    total_lines = frame_height // cell_size

    # Prepare list for storing green line y-coordinates and their corresponding numbers
    green_lines = []

    # Draw vertical lines
    for i in range(1, frame_width // cell_size):
        x = i * cell_size
        cv2.line(frame, (x, 0), (x, frame_height), (255, 0, 0), 1)  # Vertical lines in red

    # Draw horizontal lines and label green lines from bottom to top
    # Add the extra green line at the bottom with value 0
    green_lines.append((frame_height, 0))
    cv2.line(frame, (0, frame_height), (frame_width, frame_height), (0, 255, 0), 1)
    cv2.putText(frame, '0', (10, frame_height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    for i in range(total_lines):
        y = (i + 1) * cell_size
        if (i + 1) % 2 == 0:
            line_color = (0, 255, 0)  # Green for every second line
            cv2.line(frame, (0, frame_height - y), (frame_width, frame_height - y), line_color, 1)
            # Add numbering for green lines from top to bottom
            line_number = (i // 2) + 1  # Numbering from 1 to n
            green_lines.append((frame_height - y, line_number))  # Store line y-coordinate and number
        else:
            line_color = (255, 0, 0)  # Red for others
            cv2.line(frame, (0, frame_height - y), (frame_width, frame_height - y), line_color, 1)  # Horizontal lines

    # Draw black lines based on d1 values
    black_lines = []
    for d1_value in data['d1']:
        black_line_y = get_black_line_y(d1_value, green_lines, frame_height)
        black_lines.append((black_line_y, len(black_lines) + 1))
        cv2.line(frame, (0, int(black_line_y)), (frame_width, int(black_line_y)), (0, 0, 0), 1)

    # Add text labels for green lines
    for y_pos, number in green_lines:
        text_x = 10
        text_y = y_pos + 20  # Position text below the line
        cv2.putText(frame, str(number), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    # Add text labels for black lines with corrected numbering
    for y_pos, number in black_lines:
        text_x = frame_width - 50
        text_y = int(y_pos) - 10  # Position text above the line
        cv2.putText(frame, str(number), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    
    # Ensure the 0 black line label is added
    cv2.putText(frame, '0', (frame_width - 50, frame_height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    if detections.shape[0] > 0:
        # Find the largest bounding box based on area
        areas = (detections[:, 2] - detections[:, 0]) * (detections[:, 3] - detections[:, 1])
        largest_index = areas.argmax()
        largest_box = detections[largest_index]

        # Get the bottom y-coordinate of the largest bounding box
        x1, y1, x2, y2 = largest_box
        y_bottom = int(y2)

        # Draw rectangle around the largest bounding box
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        # Display the base line label in the top-left corner
        base_line_label = get_base_line_label(y_bottom, black_lines, num, opposite)
        if base_line_label:
            text_x = 10
            text_y = 30  # Position text in the top-left corner
            cv2.putText(frame, base_line_label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

        
        # Display 'FIRE' label on the opposite side of the detected fire (right hand)
        #fire_label = 'FIRE'
       # fire_text_x = int(x2) + 10
       # fire_text_y = int(y1) - 10  # Position text above the rectangle
       # cv2.putText(frame, fire_label, (fire_text_x, fire_text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame with annotations and grid
    cv2.imshow("Video Feed", frame)

    # Check for user input
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
cap.release()
cv2.destroyAllWindows()
